# <font color='blue'><div style="text-align: center">센서스 데이터 분석</font> 

<img src="https://drive.google.com/uc?id=1LDKIbPd0N8qnx0Df5BBUar4D6ZgG70nx" width="300" height="300">
<img src="https://drive.google.com/uc?id=1rRGa6DQf5Ao6_QLWkvMto1nAdS9kxiYZ" width="300" height="300">

* Q1: 총 가구수는 몇 가구인가? 
* Q2: 가구당 구성원 평균수를 시도 별로 추정하시오.
* Q3: 외국인 거주자 수와 비율을 시도 단위로 구하시오.
* Q4: 시도별 자녀 수 평균을 구하시오.  
* Q5: 시도별 자전거로 출퇴근하는 사람의 비율을 구하시오.



In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

dirName = 'D:/NaverCloud/Lecture/통계교육원/파이썬통계/data/'
census = pd.read_csv(dirName + 'census2015.csv')
f = open(dirName + "varNames.csv", 'r')
census.columns = list(f.readline().split(','))
census.head()

,gagu_ID,sido,sidosigungu,eup_dong,island_apt,gagu_no,sex,age,relation,KoreanYN,...,elder4,elder5,elder6,elder7,elder8,elder9,elder10,elder11,elder12,weight
0,1502000001,35,35030,2,1,2,1,63,3,1,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23
1,1502000002,34,34370,2,1,1,1,78,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,24
2,1502000003,31,31060,3,1,1,1,46,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56
3,1502000003,31,31060,3,1,2,2,47,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56
4,1502000004,35,35040,2,1,1,1,73,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16


In [7]:
# Q1: 총 가구수는 몇 가구인가?

len(set(census['gagu_ID']))

382217

In [16]:
# Q2: 가구당 구성원 평균수를 시도 별로 추정하시오.
groupby_sido = census.groupby('sido')
_tmp1 = groupby_sido['gagu_ID'].nunique()
_tmp2 = groupby_sido['gagu_ID'].count()
print(_tmp1, _tmp2)
_tmp2 / _tmp1

sido
11    2.494017
21    2.396333
22    2.509391
23    2.592242
24    2.509810
25    2.451306
26    2.561051
29    2.420028
31    2.666891
32    2.243924
33    2.290399
34    2.278541
35    2.237486
36    2.133165
37    2.143735
38    2.320118
39    2.524772
Name: gagu_ID, dtype: float64

In [30]:
# Q3: 외국인 거주자 수와 비율을 시도 단위로 구하시오.

_tmp = pd.crosstab(census['sido'], census['KoreanYN'], margins=True)
_tmp.columns = ['Korean','Foreigner','Total']
_tmp['ratio'] = _tmp['Foreigner'] * 100 / _tmp['Total']
_tmp

,Korean,Foreigner,Total,ratio
sido,,,,
11,152357,1464,153821,0.951756
21,57774,258,58032,0.444582
22,43271,149,43420,0.343160
23,47794,388,48182,0.805280
24,26092,133,26225,0.507150
25,25401,122,25523,0.478000
26,19663,116,19779,0.586481
29,5183,37,5220,0.708812
31,204136,1932,206068,0.937555


In [35]:
# Q4: 시도별 자녀 수 평균을 구하시오.

census['n_sons'] = census['son_male'] + census['son_female']
groupby_sido = census.groupby('sido')
groupby_sido['n_sons'].mean()

sido
11    2.127707
21    2.255253
22    2.255857
23    2.213136
24    2.488529
25    2.293818
26    2.186294
29    2.500970
31    2.202320
32    2.756340
33    2.795097
34    2.904799
35    3.161136
36    3.403941
37    2.906253
38    2.708301
39    2.647710
Name: n_sons, dtype: float64

In [39]:
pd.crosstab(census['sido'], census['byCycle'], margins=True)


byCycle,1.0,All
sido,,
11,1499,1499
21,261,261
22,493,493
23,480,480
24,196,196
25,306,306
26,324,324
29,73,73
31,1762,1762


# <font color='blue'><div style="text-align: center">SOHO 분석</font> 

* V58은 매출액 구간이고, future는 미래 폐업 여부(0: 유지, 1:폐업 혹은 전업) 임
* V7은 가족구성원 수, V8은 가족 외 구성원 수이고, emp는 종업원 중 외부인 비율임
* V10은 보증금, V11은 월세임
* 회귀모형으로 매출액을 예측해 보자

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

dirName = 'D:/NaverCloud/Lecture/통계교육원/파이썬통계/data/'
soho = pd.read_csv(dirName + 'SOHO2013.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc1 in position 0: invalid start byte